In [ ]:
!pip3 install git+https://github.com/Centre-automatique-et-systemes/learn_observe_KKL@noise gwpy &> /dev/null
!pip3 install git+https://github.com/aliutkus/torchinterp1d.git gwpy &> /dev/null


In [ ]:
import sys ; sys.path.append('../')
from learn_KKL.learner_noise import LearnerNoise
from learn_KKL.system import RevDuffing
from learn_KKL.luenberger_observer_noise import LuenbergerObserverNoise

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split

import numpy as np
import pytorch_lightning as pl
import torch.optim as optim


In [ ]:
system = RevDuffing()

w_c_arr = np.array([0.2, 0.4, 0.6, 0.8])
observer = LuenbergerObserverNoise(dim_x=2, dim_y=1, method="Supervised", wc=0.2,
                                    recon_lambda=0.8)
observer.set_dynamics(system)
data = observer.generate_data_svl(np.array([[-1., 1.], [-1., 1.]]),
                                    w_c_arr, 100000, method="LHS")

data, val_data = train_test_split(data, test_size=0.3, shuffle=True)

# Train the forward transformation using pytorch-lightning and the learner class
# Options for training
trainer_options = {'max_epochs': 0}
optimizer_options = {'weight_decay': 1e-6}
scheduler_options = {'mode': 'min', 'factor': 0.1, 'patience': 3,
                        'threshold': 1e-4, 'verbose': True}
stopper = pl.callbacks.early_stopping.EarlyStopping(
    monitor='val_loss', min_delta=5e-4, patience=3, verbose=False, mode='min')
# Instantiate learner
learner_T_star = LearnerNoise(observer=observer, system=system, training_data=data,
                            validation_data=val_data, method='T_star', batch_size=10,
                            lr=1e-3, optimizer=optim.Adam,
                            optimizer_options=optimizer_options,
                            scheduler=optim.lr_scheduler.ReduceLROnPlateau,
                            scheduler_options=scheduler_options)
# Define logger and checkpointing
logger = TensorBoardLogger(save_dir=learner_T_star.results_folder + '/tb_logs')
checkpoint_callback = ModelCheckpoint(monitor='val_loss')
trainer = pl.Trainer(
    callbacks=[stopper, checkpoint_callback], **trainer_options, logger=logger,
    log_every_n_steps=1, check_val_every_n_epoch=3)

In [ ]:
# To see logger in tensorboard, copy the following output name_of_folder
print(f'Logs stored in {learner_T_star.results_folder}/tb_logs')
# which should be similar to jupyter_notebooks/runs/method/exp_0/tb_logs/
# Then type this in terminal:
# tensorboard --logdir=name_of_folder --port=8080

# Train and save results
trainer.fit(learner_T_star)
learner_T_star.save_results(limits=np.array([[-1, 1.], [-1., 1.]]), wc_arr_train=w_c_arr,
                        nb_trajs=10, t_sim=(0, 40), dt=1e-2, checkpoint_path=checkpoint_callback.best_model_path)